<a href="https://colab.research.google.com/github/jeffersonramelo/Peper-relato-integrado-DIF-DIF/blob/main/Relato%20integrado_CVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install wget

In [17]:
import pandas as pd
import wget
from zipfile import ZipFile

In [18]:
#Acessando a base de dados e criando arquivos históricos
url_base = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/IPE/DADOS/'

In [19]:
arquivos_zip = []
for ano in range(2025, 2026):
  arquivos_zip.append(f'ipe_cia_aberta_{ano}.zip')

In [20]:
arquivos_zip

['ipe_cia_aberta_2025.zip']

In [21]:
for arq in arquivos_zip:
  wget.download(url_base+arq)

In [22]:
#extraindo os arquivos do ZIP
for arq in arquivos_zip:
  ZipFile(arq, 'r').extractall('CVM')


In [23]:
df = pd.read_csv("/content/CVM/ipe_cia_aberta_2025.csv", encoding="latin1", sep=";")

estabelecendo um filtro das variáveis que quer vizualizar e organizando pela coluna Data_Entrega

In [24]:
sorted(df['Categoria'].dropna().unique())



['Acordo de Acionistas',
 'Assembleia',
 'Ato Homologatório emitido pelo Banco Central',
 'Aviso aos Acionistas',
 'Aviso aos Debenturistas',
 'Calendário de Eventos Corporativos',
 'Carta Anual de Governança Corporativa',
 'Comunicado ao Mercado',
 'Comunicação sobre Transação entre Partes Relacionadas',
 'Comunicação sobre demandas societárias',
 'Contratos de Indenidade',
 'Convenção de grupos de sociedades',
 'Código de Conduta',
 'Dados Econômico-Financeiros',
 'Dados cadastrais de companhias incentivadas',
 'Documentos de Oferta de Distribuição Pública',
 'Emissor Estrangeiro  (entidade de investimentos)',
 'Escrituras e aditamentos de debêntures',
 'Estatuto Social',
 'Fato Relevante',
 'Informação Prestada às Bolsas Estrangeiras',
 'Informações Companhias em Falência',
 'Informações Companhias em Liquidação',
 'Informações de Companhias em Recuperação Judicial ou Extrajudicial',
 'Informações sobre acordos de acionistas previstas no art. 30, XIX, IN 480/09',
 'OPA - Edital de O

In [25]:
# Cria o DataFrame organizado
texto = (
    df[['Nome_Companhia', 'Data_Entrega', 'Categoria', 'Link_Download']]
    .drop_duplicates()
    .set_index('Data_Entrega')
)

# Filtra apenas as categorias desejadas
filtro = texto[texto['Categoria'].isin(['Relato Integrado', 'Relatório de Sustentabilidade'])]

# Salva em CSV mantendo a estrutura
filtro.to_csv('categorias_filtradas.csv', encoding='utf-8-sig')


In [26]:
import pandas as pd

# Lê o arquivo CSV
df = pd.read_csv('/content/categorias_filtradas.csv', encoding='utf-8-sig')

# Mostra as primeiras linhas para conferir
df


,Data_Entrega,Nome_Companhia,Categoria,Link_Download
0,2025-04-17,CENTRAIS ELETRICAS BRASILEIRAS SA,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
1,2025-07-11,CENTRAIS ELETRICAS BRASILEIRAS SA,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
2,2025-05-30,CARAMURU ALIMENTOS S.A.,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
3,2025-07-22,CARAMURU ALIMENTOS S.A.,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
4,2025-05-20,ENERGISA SA,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
...,...,...,...,...
213,2025-04-30,IRANI PAPEL E EMBALAGEM S.A.,Relato Integrado,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
214,2025-06-06,TRÊS TENTOS AGROINDUSTRIAL S.A.,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
215,2025-09-03,PACAEMBU CONSTRUTORA S.A.,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
216,2025-06-06,LOJAS QUERO QUERO S.A.,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...


In [27]:
!pip install PyPDF2


In [28]:
import requests
from io import BytesIO
from PyPDF2 import PdfReader

def extrair_texto_pdf(url):
    try:
        # Faz o download do arquivo
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Lê o PDF diretamente da memória
        with BytesIO(response.content) as f:
            pdf = PdfReader(f)
            texto = ""
            for pagina in pdf.pages:
                texto += pagina.extract_text() or ""
        return texto.strip()
    except Exception as e:
        return f"Erro ao processar: {e}"


In [29]:
# Cria uma nova coluna com o texto extraído
df['Texto_Extraido'] = df['Link_Download'].apply(extrair_texto_pdf)

# Exibe um exemplo
df.head(3)


,Data_Entrega,Nome_Companhia,Categoria,Link_Download,Texto_Extraido
0,2025-04-17,CENTRAIS ELETRICAS BRASILEIRAS SA,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...,RELA TÓRIOANUAL\nELETROBRAS\n 2024Sumário\nAPR...
1,2025-07-11,CENTRAIS ELETRICAS BRASILEIRAS SA,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...,RELA TÓRIOANUAL\nELETROBRAS\n 2024Sumário\nAPR...
2,2025-05-30,CARAMURU ALIMENTOS S.A.,Relatório de Sustentabilidade,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...,forçaNossa\nessência\né nossaRELATÓRIO DE \nSU...


In [30]:
df.to_csv('/content/2024.csv', encoding='utf-8-sig', index=False)
